In [ ]:
!pip install np_utils

In [ ]:
import numpy as np
import re
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.models import Model, Sequential
from keras.optimizers import RMSprop
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback
import np_utils
from keras.models import load_model, save_model
from tensorflow.keras.optimizers import Adam
import matplotlib as plt

In [ ]:
load_saved_model = True
train_model = False
seq_length = 20
filename = "/content/drive/MyDrive/rockyou-75.txt"


In [ ]:
with open(filename, encoding='utf-8-sig') as f:
    text = f.read().lower().strip()

chars = sorted(list(set(text)))
total_chars = len(chars)
print("Unique characters in the text:", total_chars)

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


In [ ]:
X = np.zeros((len(text) - seq_length, seq_length, total_chars), dtype=bool)
y = np.zeros((len(text) - seq_length, total_chars), dtype=bool)

for i in range(len(text) - seq_length):
    for t, char in enumerate(text[i:i + seq_length]):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[text[i + seq_length]]] = 1


In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, diversity):
    start_index = np.random.randint(0, len(text) - seq_length - 1)
    sentence = text[start_index: start_index + seq_length]
    generated = sentence
    for i in range(length):
        x_pred = np.zeros((1, seq_length, total_chars))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
    return generated


In [ ]:
if load_saved_model:
    model = load_model('/content/drive/MyDrive/model-saveLSTM.h5')
else:
    model = Sequential()
    model.add(LSTM(256, input_shape=(seq_length, total_chars), return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(total_chars, activation='softmax'))
    optimizer = Adam(learning_rate=0.00001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [ ]:
if train_model:
    history = model.fit(X, y, batch_size=32, epochs=100)
    
    model_save_path = '/content/drive/MyDrive/model-saveLSTM.h5'
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    plt.plot(history.history['loss'])
    plt.title('Model Loss Over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    loss_plot_path = '/content/drive/MyDrive/model_loss_plot.png'
    plt.savefig(loss_plot_path)
    print(f"Loss plot saved to {loss_plot_path}")
    plt.show()


In [ ]:
generated_text = generate_text(200, 4)
print(generated_text)

with open('LSTM-output.txt', 'w') as file:
    file.write(generate_text(1000, 0.5))
